In [1]:
import pandas as pd

In [2]:
public_df = pd.read_csv('D:Tbrain_31/dataset_1st/public_processed.csv')
train_df = pd.read_csv('D:Tbrain_31/dataset_1st/training.csv')
### 保留 txkey 欄位最後上傳用
final_df = public_df[['txkey']]

### 預測用資料集，訓練完模型在跑這行就好
## 把資料轉成正確data type
# 類別變數比較多，所以先把全部轉成類別
public_df = public_df.astype('category')

# 剩下轉回數值變數
public_df[['locdt', 'loctm', 'flam1', 'csmam']] = public_df[['locdt', 'loctm', 'flam1', 'csmam']].astype('int64')
public_df[['conam', 'iterm']] = public_df[['conam', 'iterm']].astype('float64')

## 缺失值填充

# 指定要填充的欄位
columns_to_fill = ['etymd', 'mcc', 'stocn', 'scity', 'hcefg', 'csmcu']

# 將 "others" 添加到類別中，如果已經存在，則忽略
for column in columns_to_fill:
    public_df[column] = public_df[column].cat.add_categories('-1').fillna('-1')


# stscd(狀態碼)幾乎全部都是缺失值，也應該不是重要特徵，先就刪掉這欄
public_df.drop('stscd', axis=1, inplace=True)

# txkey全部都是唯一資料，刪除不用
public_df.drop('txkey', axis=1, inplace=True)

## 把資料轉成正確data type
# 類別變數比較多，所以先把全部轉成類別
train_df = train_df.astype('category')

# 剩下轉回數值變數
train_df[['locdt', 'loctm', 'flam1', 'csmam']] = train_df[['locdt', 'loctm', 'flam1', 'csmam']].astype('int64')
train_df[['conam', 'iterm']] = train_df[['conam', 'iterm']].astype('float64')


## 缺失值填充

# 指定要填充的欄位
columns_to_fill = ['etymd', 'mcc', 'stocn', 'scity', 'hcefg', 'csmcu']

# 將 "others" 添加到類別中，如果已經存在，則忽略
for column in columns_to_fill:
    train_df[column] = train_df[column].cat.add_categories('-1').fillna('-1')


# stscd(狀態碼)幾乎全部都是缺失值，也應該不是重要特徵，先就刪掉這欄
train_df.drop('stscd', axis=1, inplace=True)

### 移除不必要的column

## 檢查txkey是否只包含唯一值
#uni_txkey = train_df['txkey'].value_counts().reset_index()
#print(uni_txkey.loc[uni_txkey['count'] != 1])

# txkey全部都是唯一資料，刪除不用
train_df.drop('txkey', axis=1, inplace=True)

## PyCaret Tutorial

In [3]:
# label轉成數值，不轉會出現error
train_df['label'] = train_df['label'].astype('int64')

In [4]:
# import ClassificationExperiment and init the class
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

In [12]:
categorical_feature = [col for col in train_df.columns if train_df[col].dtype == 'category' and col != 'label']
print(categorical_feature)
print('Number of categorical feature:', len(categorical_feature))

['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic', 'mcc', 'ecfg', 'insfg', 'bnsfg', 'stocn', 'scity', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']
Number of categorical feature: 17


### Setup

In [18]:
# init setup on exp
exp.setup(train_df, target='label',
          fix_imbalance=True, fix_imbalance_method='RandomOverSampler',
          n_jobs=10,
          fold=5,
          log_experiment=True, experiment_name='01_FirstExp',
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(8688526, 24)"
4,Transformed data shape,"(14725654, 51)"
5,Transformed train set shape,"(12119096, 51)"
6,Transformed test set shape,"(2606558, 51)"
7,Ordinal features,6
8,Numeric features,6
9,Categorical features,17


2023/11/14 11:00:33 INFO mlflow.tracking.fluent: Experiment with name '01_FirstExp' does not exist. Creating a new experiment.


In [7]:
exp.models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [8]:
# compare selected models
include_models = ['lr', 'nb', 'dt', 'svm', 'mlp', 'rf', 'lightgbm'] 
best = exp.compare_models(include=include_models, sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9987,0.9627,0.7514,0.8818,0.8092,0.8086,0.8123,44.0080
rf,Random Forest Classifier,0.9987,0.8763,0.6872,0.9608,0.8013,0.8006,0.8120,269.4440
dt,Decision Tree Classifier,0.9986,0.8407,0.6817,0.9082,0.7770,0.7763,0.7853,64.1960
mlp,MLP Classifier,0.9374,0.7014,0.3857,0.0582,0.0706,0.0652,0.1028,1054.0700
nb,Naive Bayes,0.9868,0.6596,0.0612,0.0226,0.0330,0.0277,0.0313,42.9720
lr,Logistic Regression,0.9034,0.6499,0.3114,0.0120,0.0232,0.0162,0.0448,55.5600
svm,SVM - Linear Kernel,0.4609,0.0000,0.6183,0.0050,0.0100,0.0027,0.0112,174.1180


Traceback (most recent call last):
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 271, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 405, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1109, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1102, in _read_helper
    result = read_yaml(root, file_name)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\utils\file_utils.py", line 183, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist."

In [9]:
#exp.evaluate_model(best)
#exp.plot_model(best, plot='feature')

In [10]:
exp.save_model(best, '01_first_model_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['locdt', 'loctm', 'conam', 'iterm',
                                              'flam1', 'csmam'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated'))),
                 ('categorical_imputer',
                  TransformerWra...
                  LGBMClassifier(boosting_type='gbdt', class_weight

In [11]:
tune_best = exp.tune_model(best, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9988,0.9900,0.7058,0.9434,0.8075,0.8069,0.8154
1,0.9991,0.9872,0.8127,0.9426,0.8728,0.8724,0.8748
2,0.9988,0.9881,0.7183,0.9363,0.8130,0.8124,0.8196
3,0.9986,0.9851,0.6784,0.9360,0.7867,0.7860,0.7963
4,0.9987,0.9910,0.7014,0.9413,0.8038,0.8032,0.8120
Mean,0.9988,0.9883,0.7233,0.9399,0.8168,0.8162,0.8236
Std,0.0002,0.0021,0.0465,0.0031,0.0294,0.0295,0.0268


Fitting 5 folds for each of 10 candidates, totalling 50 fits


Traceback (most recent call last):
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 271, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 405, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1109, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1102, in _read_helper
    result = read_yaml(root, file_name)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\utils\file_utils.py", line 183, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist."

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


In [12]:
tune_best

LGBMClassifier(bagging_fraction=0.9, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=6, min_child_weight=0.001, min_split_gain=0.3,
               n_estimators=20, n_jobs=10, num_leaves=150, objective=None,
               random_state=123, reg_alpha=0.005, reg_lambda=0.0005,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [13]:
exp.save_model(tune_best, '02_tuned_model_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['locdt', 'loctm', 'conam', 'iterm',
                                              'flam1', 'csmam'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated'))),
                 ('categorical_imputer',
                  TransformerWra...
                                 boosting_type='gbdt', class_weight

# 新增測試

# Finalize model and make first predictions

In [4]:
import pandas as pd

public_df = pd.read_csv('/Tbrain_31/dataset_1st/public_processed.csv')
train_df = pd.read_csv('/Tbrain_31/dataset_1st/training.csv')
### 保留 txkey 欄位最後上傳用
final_df = public_df[['txkey']]

### 預測用資料集，訓練完模型在跑這行就好
## 把資料轉成正確data type
# 類別變數比較多，所以先把全部轉成類別
public_df = public_df.astype('category')

# 剩下轉回數值變數
public_df[['locdt', 'loctm', 'flam1', 'csmam']] = public_df[['locdt', 'loctm', 'flam1', 'csmam']].astype('int64')
public_df[['conam', 'iterm']] = public_df[['conam', 'iterm']].astype('float64')

## 缺失值填充

# 指定要填充的欄位
columns_to_fill = ['etymd', 'mcc', 'stocn', 'scity', 'hcefg', 'csmcu']

# 將 "others" 添加到類別中，如果已經存在，則忽略
for column in columns_to_fill:
    public_df[column] = public_df[column].cat.add_categories('-1').fillna('-1')


# stscd(狀態碼)幾乎全部都是缺失值，也應該不是重要特徵，先就刪掉這欄
public_df.drop('stscd', axis=1, inplace=True)

# txkey全部都是唯一資料，刪除不用
public_df.drop('txkey', axis=1, inplace=True)

## 把資料轉成正確data type
# 類別變數比較多，所以先把全部轉成類別
train_df = train_df.astype('category')

# 剩下轉回數值變數
train_df[['locdt', 'loctm', 'flam1', 'csmam']] = train_df[['locdt', 'loctm', 'flam1', 'csmam']].astype('int64')
train_df[['conam', 'iterm']] = train_df[['conam', 'iterm']].astype('float64')


## 缺失值填充

# 指定要填充的欄位
columns_to_fill = ['etymd', 'mcc', 'stocn', 'scity', 'hcefg', 'csmcu']

# 將 "others" 添加到類別中，如果已經存在，則忽略
for column in columns_to_fill:
    train_df[column] = train_df[column].cat.add_categories('-1').fillna('-1')


# stscd(狀態碼)幾乎全部都是缺失值，也應該不是重要特徵，先就刪掉這欄
train_df.drop('stscd', axis=1, inplace=True)

### 移除不必要的column

## 檢查txkey是否只包含唯一值
#uni_txkey = train_df['txkey'].value_counts().reset_index()
#print(uni_txkey.loc[uni_txkey['count'] != 1])

# txkey全部都是唯一資料，刪除不用
train_df.drop('txkey', axis=1, inplace=True)

# label轉成數值，不轉會出現error
train_df['label'] = train_df['label'].astype('int64')

# import ClassificationExperiment and init the class
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

categorical_feature = [col for col in train_df.columns if train_df[col].dtype == 'category' and col != 'label']
#print(categorical_feature)
#print('Number of categorical feature:', len(categorical_feature))

# init setup on exp
exp.setup(train_df, target='label',
          fix_imbalance=True, fix_imbalance_method='RandomOverSampler',
          n_jobs=10,
          fold=5,
          log_experiment=True, experiment_name='01_FirstExp',
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(8688526, 24)"
4,Transformed data shape,"(14725654, 51)"
5,Transformed train set shape,"(12119096, 51)"
6,Transformed test set shape,"(2606558, 51)"
7,Ordinal features,6
8,Numeric features,6
9,Categorical features,17


Traceback (most recent call last):
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 271, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 405, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1109, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1102, in _read_helper
    result = read_yaml(root, file_name)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\utils\file_utils.py", line 183, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist."

## tune_and_caliberate_model_for_the first_trail_lgbm

In [5]:
tune_model = exp.load_model('02_tuned_model_pipeline')

Transformation Pipeline and Model Successfully Loaded


In [6]:
tune_model = exp.load_model('02_tuned_model_pipeline')
best_threshold_model = exp.optimize_threshold(tune_model, optimize='F1')
exp.save_model(best_threshold_model, '03_tuned_thresholdOptimize_pipeline')

Transformation Pipeline and Model Successfully Loaded


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\pycaret\internal\pipeline.py", line 275, in fit
    fitted_estimator = self._memory_fit(
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\joblib\memory.py", line 353, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\pycaret\internal\pipeline.py", line 68, in _fit_one
    transformer.fit(*args, **fit_param

In [ ]:
finalize_bst_threshold_model = exp.finalize_model(best_threshold_model)
exp.save_model(finalize_bst_threshold_model, '04_finalized_bst_threshold_pipeline')

In [ ]:
bst_threshold_tuned_caliberate_model = exp.calibrate_model(best_threshold_model)
exp.save_model(bst_threshold_tuned_caliberate_model, '05_all_optimize_func_pipeline')

In [ ]:
finalize_bst_threshold_caliberated_model = exp.finalize_model(bst_threshold_tuned_caliberate_model)
exp.save_model(finalize_bst_threshold_caliberated_model, '06_finalized_lgbm_threshold_caliberate_pipeline')

# AutoML run trial

In [7]:
# init setup on exp
exp.setup(train_df, target='label',
          fix_imbalance=True, fix_imbalance_method='RandomOverSampler',
          n_jobs=10,
          fold=5,
          log_experiment=True, experiment_name='02_automlExp',
          session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(8688526, 24)"
4,Transformed data shape,"(14725654, 51)"
5,Transformed train set shape,"(12119096, 51)"
6,Transformed test set shape,"(2606558, 51)"
7,Ordinal features,6
8,Numeric features,6
9,Categorical features,17


Traceback (most recent call last):
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 271, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 405, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1109, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\store\tracking\file_store.py", line 1102, in _read_helper
    result = read_yaml(root, file_name)
  File "C:\Users\vghuser\anaconda3\envs\credit_ML\lib\site-packages\mlflow\utils\file_utils.py", line 183, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist."

In [8]:
#tune_lgbm_model = exp.load_model('02_tuned_model_pipeline')

In [9]:
### 因為之前load的模型(tune_lgbm_model: 02_tuned_model_pipeline)不讓我直接blend
## 所以利用其儲存的超參數重新創建一個lightgbm模型

# 定義 LightGBM 的超參數
lgbm_params = {
    'bagging_fraction': 0.9,
    'bagging_freq': 3,
    'feature_fraction': 0.5,
    'learning_rate': 0.4,
    'min_child_samples': 6,
    'min_split_gain': 0.3,
    'n_estimators': 20,
    'n_jobs': 10,
    'num_leaves': 150,
    'random_state': 123,
    'reg_alpha': 0.005,
    'reg_lambda': 0.0005
}

# 創建 LightGBM 模型
tune_lgbm_model = exp.create_model('lightgbm', **lgbm_params)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9988,0.9900,0.7058,0.9434,0.8075,0.8069,0.8154
1,0.9991,0.9872,0.8127,0.9426,0.8728,0.8724,0.8748
2,0.9988,0.9881,0.7183,0.9363,0.8130,0.8124,0.8196
3,0.9986,0.9851,0.6784,0.9360,0.7867,0.7860,0.7963
4,0.9987,0.9910,0.7014,0.9413,0.8038,0.8032,0.8120
Mean,0.9988,0.9883,0.7233,0.9399,0.8168,0.8162,0.8236
Std,0.0002,0.0021,0.0465,0.0031,0.0294,0.0295,0.0268


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


In [10]:
rf_model = exp.create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9988,0.8749,0.6922,0.9625,0.8053,0.8047,0.8157
1,0.9988,0.8806,0.6927,0.9589,0.8044,0.8037,0.8145
2,0.9987,0.8885,0.6844,0.9552,0.7975,0.7968,0.8080
3,0.9987,0.8654,0.6740,0.9600,0.7919,0.7913,0.8038
4,0.9988,0.8722,0.6927,0.9673,0.8073,0.8067,0.8180
Mean,0.9987,0.8763,0.6872,0.9608,0.8013,0.8006,0.8120
Std,0.0000,0.0078,0.0073,0.0040,0.0057,0.0057,0.0053


In [11]:
tune_rf_model = exp.tune_model(rf_model, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9976,0.8564,0.7141,0.6616,0.6868,0.6856,0.6861
1,0.9973,0.8651,0.7322,0.6126,0.6671,0.6657,0.6684
2,0.9971,0.8588,0.7197,0.5906,0.6488,0.6473,0.6505
3,0.9973,0.8448,0.6909,0.6221,0.6547,0.6533,0.6543
4,0.9975,0.8565,0.7141,0.6486,0.6798,0.6785,0.6793
Mean,0.9974,0.8563,0.7142,0.6271,0.6674,0.6661,0.6677
Std,0.0002,0.0066,0.0134,0.0254,0.0144,0.0145,0.0138


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [12]:
blend_rf_lgbm = exp.blend_models(['tune_lgbm_model', 'tune_rf_model'], optimize='F1', return_train_score=True)

ValueError: Estimator tune_lgbm_model does not have the required fit() method.

In [ ]:
stack_rf_lgbm = exp.stack_models(['tune_lgbm_model', 'tune_rf_model'], optimize='F1', return_train_score=True)

In [ ]:
best_f1_model_by_automl = exp.automl(optimize='F1')

In [ ]:
exp.save_model(tune_rf_model, '11_tuned_rf_model')
exp.save_model(blend_rf_lgbm, '12_blend_rf_lgbm_model')
exp.save_model(stack_rf_lgbm, '13_stack_rf_lgbm_model')
exp.save_model(best_f1_model_by_automl, '14_best_f1_model_by_automl')

In [ ]:
best_optimize_threshold = exp.optimize_threshold(best_f1_model_by_automl, optimize='F1')

In [ ]:
best_optimize_threshold_caliberated = exp.calibrate_model(best_optimize_threshold, return_train_score=True)

In [ ]:
exp.save_model(best_optimize_threshold, '15_best_optimize_threshold')
exp.save_model(best_optimize_threshold_caliberated, '16_best_optimize_threshold_caliberated')

In [ ]:
bst_by_automl_optimize_threshold_finalize = exp.finalize_model(best_optimize_threshold)

In [ ]:
bst_threshold_caliberate = exp.finalize_model(best_optimize_threshold_caliberated)

In [ ]:
exp.save_model(bst_by_automl_optimize_threshold_finalize, '16_bst_by_automl_optimize_threshold_finalize')
exp.save_model(bst_threshold_caliberate, '17_bst_threshold_caliberate')